In [1]:
import sys
import os
import numpy as np
import dill
import gzip
import logging


sys.path.append(os.path.abspath('../../surmise/emulationmethods'))
from AKSGP import Emulator

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        # logging.FileHandler('emulator_train.log', mode='w'),  # Log to file
        logging.StreamHandler()  # Log to console
    ]
)
logger = logging.getLogger(__name__)

# Load data

In [2]:
# Load training data
train_dir = 'simulation_data/Grad_Pb-Pb-2760GeV/train'
X = np.loadtxt(os.path.join(train_dir, 'X.txt'))
Ymean = np.loadtxt(os.path.join(train_dir, 'Ymean.txt'))
Ystd = np.loadtxt(os.path.join(train_dir, 'Ystd.txt'))

print(f"Arrays loaded from directory '{train_dir}'.")
print(f"Shapes of loaded arrays: {X.shape}, {Ymean.shape}, {Ystd.shape}")


# Load testing data
test_dir = 'simulation_data/Grad_Pb-Pb-2760GeV/test'

Xval = np.loadtxt(os.path.join(test_dir, 'X.txt'))
Ymeanval = np.loadtxt(os.path.join(test_dir, 'Ymean.txt'))
Ystdval = np.loadtxt(os.path.join(test_dir, 'Ystd.txt'))

print(f"Arrays loaded from directory '{test_dir}'.")
print(f"Shapes of loaded arrays: {Xval.shape}, {Ymeanval.shape}, {Ystdval.shape}")


Arrays loaded from directory 'simulation_data/Grad_Pb-Pb-2760GeV/train'.
Shapes of loaded arrays: (485, 17), (485, 110), (485, 110)
Arrays loaded from directory 'simulation_data/Grad_Pb-Pb-2760GeV/test'.
Shapes of loaded arrays: (93, 17), (93, 110), (93, 110)


# Train emulators

In [3]:
inpsize = 485; numobs= (0,4,7,34);

emu = Emulator(X=X[:inpsize], Y_mean=Ymean[:inpsize, numobs], Y_std=Ystd[:inpsize, numobs])
emu.fit(kernel='AKS', nrestarts=5, seed=13)


2024-08-23 16:20:31,953 - AKSGP - INFO - Automatic kernel selection opted. Best kernel for each output dimension will be selected from the list of kernels:
   ['constom_kernel', 'Matern12', 'Matern32', 'Matern52', 'RBF', 'DotProduct+Matern12', 'DotProduct+Matern32', 'DotProduct+Matern52', 'DotProduct+RBF', 'DotProduct*Matern12', 'DotProduct*Matern32', 'DotProduct*Matern52', 'DotProduct*RBF', 'ExpSineSquared+Matern12', 'ExpSineSquared+Matern32', 'ExpSineSquared+Matern52', 'ExpSineSquared+RBF', 'ExpSineSquared*Matern12', 'ExpSineSquared*Matern32', 'ExpSineSquared*Matern52', 'ExpSineSquared*RBF', 'RationalQuadratic+Matern12', 'RationalQuadratic+Matern32', 'RationalQuadratic+Matern52', 'RationalQuadratic+RBF', 'RationalQuadratic*Matern12', 'RationalQuadratic*Matern32', 'RationalQuadratic*Matern52', 'RationalQuadratic*RBF']

2024-08-23 16:20:31,953 - AKSGP - INFO - Shape of training arrays: (436, 17), (436, 4), (436, 4)
2024-08-23 16:20:31,953 - AKSGP - INFO - Shape of pseudo_test arrays: (

InvalidParameterError: The 'kernel' parameter of GaussianProcessRegressor must be None or an instance of 'sklearn.gaussian_process.kernels.Kernel'. Got <AKSGP.CustomKernel object at 0x10b671d90> instead.

In [ ]:
test = 10

print("Predictions on training set ----------------->")
# Predict with standard deviation
means, std_devs = emu.predict(X[:test])

print("  Original means:\n", Ymean[:test, numobs],"\n")
print("  Predicted means:\n", means,"\n")

print("  Original standard deviations:\n", Ystd[:test, numobs],"\n")
print("  Predicted standard deviations:\n", std_devs,"\n")


print("Predictions on test set ----------------->")
# Predict with standard deviation
means, std_devs = emu.predict(Xval[:test])

print("  Original means:\n", Ymeanval[:test, numobs],"\n")
print("  Predicted means:\n", means,"\n")

print("  Original standard deviations:\n", Ystdval[:test, numobs],"\n")
print("  Predicted standard deviations:\n", std_devs,"\n")
